In [1]:
import os 
import sys

import FINE as fn
import FINE.IOManagement.xarrayIO as xrIO 

# %load_ext autoreload
# %autoreload 2

# How to save an energy system model instance and set it back up? 

**Xarray and NetCDF files to the rescue!** The data contained within an Energy System Model (ESM) instance is vast and complex. Saving it directly is not possible. It can, however, be saved as a NetCDF file which supports complex data structures. 

#### What exactly is NetCDF? 
NetCDF (Network Common Data Format) is a set of software libraries and machine-independent data formats that support the creation, access, and sharing of array-oriented scientific data. It is also a community standard for sharing scientific data. 

#### Python modules that support working with NetCDF files:
1. netcdf4-python: Official Python interface to netCDF files
2. PyNIO: To access different file formats such as netCDF, HDF, and GRIB
3. xarray: Based on NumPy and pandas

Note: xarray module is used here. 

For our use case, the following functionalities are provided: 
* Conversion of ESM instance to xarray dataset. Additionally, possible to save this dataset as NetCDF file in a desired folder, with a desired file name. 
* Conversion of xarray dataset/saved NetCDF file back to ESM instance.

#### Structure of the xarray dataset: 

<img src="xarray_fine.png" style="width: 1000px;"/>


## Conversion of ESM instance to xarray dataset and saving it as a NetCDF file

#### STEP 1. Set up your  ESM instance 

In [2]:
from getModel import getModel

esM = getModel()
esM.aggregateTemporally(numberOfTypicalPeriods=4)
esM.optimize(timeSeriesAggregation=True, optimizationSpecs='OptimalityTol=1e-3 method=2 cuts=0 MIPGap=5e-3')

The distances of a component are set to a normalized value of 1.


 c:\users\k.knosala\tools\fine\FINE\utils.py:669: UserWarning:A declaration of bigM is not necessary if hasIsBuiltBinaryVariable is set to false. The value of bigM will be ignored in the optimization.



Clustering time series data with 4 typical periods and 24 time steps per period...
		(5.3904 sec)

Time series aggregation specifications:
Number of typical periods:4, number of time steps per period:24

Declaring sets, variables and constraints for SourceSinkModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.8504 sec)

Declaring sets, variables and constraints for ConversionModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.1487 sec)

Declaring sets, variables and constraints for StorageModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(4.4800 sec)

Declaring sets, variables and constraints for LOPFModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.3713 sec)

Declaring sets, variables and constraints for TransmissionModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.4640 sec)

Declaring shared potential constraint...
		(0.0070 s

 c:\users\k.knosala\tools\fine\FINE\storage.py:1170: UserWarning:Charge and discharge at the same time for component Salt caverns (hydrogen)


for StorageModel ...       (5.9710sec)
for LOPFModel ...          (6.1804sec)
for TransmissionModel ...  (5.1105sec)
		(21.0100 sec)



#### STEP 2. Conversion to xarray datasets and saving as NetCDF file
You can convert the esM to xarray datasets with `esm_to_datasets` and access Input, Parameters or Result.


In [4]:
esm_datasets = xrIO.esm_to_datasets(esM)

	Processing SourceSinkModel ...
	Processing ConversionModel ...
	Processing StorageModel ...
	Processing LOPFModel ...
	Processing TransmissionModel ...


 c:\users\k.knosala\tools\fine\FINE\IOManagement\utilsIO.py:70: UserWarning:Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
 c:\users\k.knosala\tools\fine\FINE\IOManagement\utilsIO.py:70: UserWarning:Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
 c:\users\k.knosala\tools\fine\FINE\IOManagement\utilsIO.py:70: UserWarning:Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
 c:\users\k.knosala\tools\fine\FINE\IOManagement\utilsIO.py:70: UserWarning:Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
 c:\users\k.knosala\tools\fine\FINE\IOManagement\utilsIO.py:37: UserWarning:More than one _ foun

In [29]:
# xr_dataset['ts_operationRateMax'].loc['Source, Wind (onshore)', :, :]
esm_datasets["Input"]["Source"]["Wind (onshore)"]["ts_operationRateMax"].to_dataframe().unstack()

ts_operationRateMax                                                    \
space           cluster_0 cluster_1 cluster_2 cluster_3 cluster_4 cluster_5   
time                                                                          
0                0.569672  0.984958  0.987930  0.851416  0.276660  0.758013   
1                0.582094  0.972450  0.976257  0.853392  0.275148  0.757377   
2                0.608804  0.951505  0.936413  0.848352  0.286690  0.754470   
3                0.637364  0.913621  0.887258  0.833918  0.303492  0.755537   
4                0.659234  0.874988  0.822736  0.817766  0.324501  0.769687   
...                   ...       ...       ...       ...       ...       ...   
8755             0.313734  0.673692  0.757970  0.343864  0.155183  0.320427   
8756             0.279877  0.642817  0.740666  0.289150  0.119844  0.294879   
8757             0.270794  0.589719  0.789207  0.272159  0.101342  0.273980   
8758             0.266642  0.544460  0.791587  0.252441  0.093761  0.256921   
8759             0.284955  0.492170  0.676788  0.270208  0.101512  0.291896   

                           
space cluster_6 cluster_7  
time                       
0      0.989970  0.623974  
1      0.974187  0.667338  
2      0.947058  0.696272  
3      0.901716  0.705269  
4      0.834311  0.710336  
...         ...       ...  
8755   0.724774  0.426975  
8756   0.672460  0.429161  
8757   0.594176  0.507018  
8758   0.520987  0.396641  
8759   0.492161  0.313036  

[8760 rows x 8 columns]

In [32]:
esm_datasets["Results"]["SourceSinkModel"]["Wind (onshore)"]

<xarray.Dataset>
Dimensions:                    (space: 8, time: 8760)
Coordinates:
  * space                      (space) <U9 'cluster_0' ... 'cluster_7'
  * time                       (time) int64 0 1 2 3 4 ... 8756 8757 8758 8759
Data variables:
    TAC                        (space) float64 6.041 0.4332 3.337 ... 6.028 0.0
    capacity                   (space) float64 45.07 3.232 24.89 ... 44.97 0.0
    capexCap                   (space) float64 5.049 0.3621 2.789 ... 5.039 0.0
    invest                     (space) float64 49.58 3.555 27.38 ... 49.47 0.0
    operation                  (space) float64 7.07e+04 7.237e+03 ... 0.0
    opexCap                    (space) float64 0.9915 0.0711 ... 0.9894 0.0
    operationVariablesOptimum  (time, space) float64 29.92 1.255 ... 20.63 nan
    capacityVariablesOptimum   (space) float64 45.07 3.232 24.89 ... 44.97 0.0

In [33]:
esm_datasets["Parameters"]

<xarray.Dataset>
Dimensions:  ()
Data variables:
    *empty*
Attributes:
    locations:           {'cluster_5', 'cluster_2', 'cluster_4', 'cluster_3',...
    commodities:         {'electricity', 'CO2', 'biogas', 'hydrogen', 'methane'}
    commodityUnitsDict:  {'electricity': 'GW$_{el}$', 'methane': 'GW$_{CH_{4}...
    numberOfTimeSteps:   8760
    hoursPerTimeStep:    1
    costUnit:            1e9 Euro
    lengthUnit:          km
    verboseLogLevel:     0
    balanceLimit:        None
    lowerBound:          False

Or save it directly to NetCDF with `esm_to_netcdf`:

In [15]:

_ = xrIO.esm_to_netcdf(esM, outputFileName="my_esm.nc")


Writing output to netCDF... 
	Processing SourceSinkModel ...
	Processing ConversionModel ...
	Processing StorageModel ...
	Processing LOPFModel ...
	Processing TransmissionModel ...


 c:\users\k.knosala\tools\fine\FINE\IOManagement\utilsIO.py:37: UserWarning:More than one _ found in series index.             Therefore, _ is not used to split the index
 c:\users\k.knosala\tools\fine\FINE\IOManagement\utilsIO.py:37: UserWarning:More than one _ found in series index.             Therefore, _ is not used to split the index
 c:\users\k.knosala\tools\fine\FINE\IOManagement\utilsIO.py:37: UserWarning:More than one _ found in series index.             Therefore, _ is not used to split the index
 c:\users\k.knosala\tools\fine\FINE\IOManagement\utilsIO.py:37: UserWarning:More than one _ found in series index.             Therefore, _ is not used to split the index
 c:\users\k.knosala\tools\fine\FINE\IOManagement\utilsIO.py:37: UserWarning:More than one _ found in series index.             Therefore, _ is not used to split the index
 c:\users\k.knosala\tools\fine\FINE\IOManagement\utilsIO.py:37: UserWarning:More than one _ found in series index.             Therefore, _ is no

Done. (15.1016 sec)


#### STEP 3. Load esM from NetCDF file or xarray datasets

You can load an esM from file with `netcdf_to_esm`.

In [24]:
# esm_from_file = xrIO.netcdf_to_esm("my_esm.nc")  # Not implemented

In [ ]:
esm_from_file.getComponentAttribute('Wind (onshore)', 'operationRateMax')

Or from datasets with `datasets_to_esm`.

In [21]:
# Alternative to giving an xr dataset, you could pass the full path to your NETCDF file 
esm_from_datasets = xrIO.datasets_to_esm(esm_datasets)

 c:\users\k.knosala\tools\fine\FINE\IOManagement\utilsIO.py:70: UserWarning:Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
 c:\users\k.knosala\tools\fine\FINE\IOManagement\utilsIO.py:70: UserWarning:Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
 c:\users\k.knosala\tools\fine\FINE\IOManagement\utilsIO.py:70: UserWarning:Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
 c:\users\k.knosala\tools\fine\FINE\IOManagement\utilsIO.py:70: UserWarning:Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
 c:\users\k.knosala\tools\fine\FINE\utils.py:669: UserWarning:A declaration of bigM is not neces

In [25]:
esm_from_datasets.getComponentAttribute('Wind (onshore)', 'operationRateMax')

space,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7
time,,,,,,,,
0,0.569672,0.984958,0.987930,0.851416,0.276660,0.758013,0.989970,0.623974
1,0.582094,0.972450,0.976257,0.853392,0.275148,0.757377,0.974187,0.667338
2,0.608804,0.951505,0.936413,0.848352,0.286690,0.754470,0.947058,0.696272
3,0.637364,0.913621,0.887258,0.833918,0.303492,0.755537,0.901716,0.705269
4,0.659234,0.874988,0.822736,0.817766,0.324501,0.769687,0.834311,0.710336
...,...,...,...,...,...,...,...,...
8755,0.313734,0.673692,0.757970,0.343864,0.155183,0.320427,0.724774,0.426975
8756,0.279877,0.642817,0.740666,0.289150,0.119844,0.294879,0.672460,0.429161
8757,0.270794,0.589719,0.789207,0.272159,0.101342,0.273980,0.594176,0.507018


In [35]:
esm_datasets["Results"]["SourceSinkModel"]["Wind (onshore)"]

<xarray.Dataset>
Dimensions:                    (space: 8, time: 8760)
Coordinates:
  * space                      (space) <U9 'cluster_0' ... 'cluster_7'
  * time                       (time) int64 0 1 2 3 4 ... 8756 8757 8758 8759
Data variables:
    TAC                        (space) float64 6.041 0.4332 3.337 ... 6.028 0.0
    capacity                   (space) float64 45.07 3.232 24.89 ... 44.97 0.0
    capexCap                   (space) float64 5.049 0.3621 2.789 ... 5.039 0.0
    invest                     (space) float64 49.58 3.555 27.38 ... 49.47 0.0
    operation                  (space) float64 7.07e+04 7.237e+03 ... 0.0
    opexCap                    (space) float64 0.9915 0.0711 ... 0.9894 0.0
    operationVariablesOptimum  (time, space) float64 29.92 1.255 ... 20.63 nan
    capacityVariablesOptimum   (space) float64 45.07 3.232 24.89 ... 44.97 0.0